<h1>Let's scrap an URL</h1>

In [44]:
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

# parse the HTML page
soup = BeautifulSoup(page, "lxml")

htmlContent = soup.prettify()
table=soup.find("table")


<h1>Let's extract the table rows<h1>

In [50]:
Postalcode      = []
Borough       = []
Neighbourhood = []
skipped = 0
# Extract individual rows and populate the lists
for cell in table.find_all('td'):
    value1 = cell.find('b')

    value2 = cell.find('span')
    # Skip the cell which is Not Assigned or doesn't end with ')' ==> Borough not known
    if ((value2.text.find('Not assigned') != -1) or (value2.text.find(')') == -1)): 
         skipped = skipped + 1
    else:
        Postalcode.append(value1.text)
        linked_all = value2.find_all("a")
        separateList = value2.text.split('(')
        Borough.append(separateList[0])
        neigh = separateList[1].replace('/', ',').replace(')', '')
        
        Neighbourhood.append(neigh)
        
print("skipped:  ", skipped)

#Create the postal code directory
toronto_postal = {'Postcode':Postalcode, 'Borough':Borough, 'Neighbourhood':Neighbourhood}
df_toronto_postal = pd.DataFrame.from_dict(toronto_postal)
df_toronto_postal.head(20)

skipped:   78


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M9A,Etobicoke,Islington Avenue
5,M1B,Scarborough,"Malvern , Rouge"
6,M3B,North York,Don MillsNorth
7,M4B,East York,"Parkview Hill , Woodbine Gardens"
8,M5B,Downtown Toronto,"Garden District, Ryerson"
9,M6B,North York,Glencairn


In [51]:
df_toronto_postal.shape

(102, 3)

<h1>Time to import the Geospatial Data</hi>

In [54]:
import pandas as pd
geoData_df = pd.read_csv('https://cocl.us/Geospatial_data')
geoData_df.head(20)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


<h1>We find the matching coordinates and append to the data frame</h1>

In [58]:
df_toronto_postal.merge(geoData_df, left_on='Postcode', right_on='Postal Code').drop(['Postal Code'], axis=1)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
5,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
6,M3B,North York,Don MillsNorth,43.745906,-79.352188
7,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
8,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
9,M6B,North York,Glencairn,43.709577,-79.445073
